In [95]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from newspaper import Article
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import pandas as pd
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

nltk.download('vader_lexicon')
nlp = en_core_web_sm.load()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nelly.loh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/nelly.loh/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [96]:
lee_dict = {'name': 'lee yan ru', 'age': '24', 'occupation': 'student'}


In [97]:
new_words = {
    'crushes': 10,
    'beats': 5,
    'misses': -5,
    'trouble': -10,
    'falls': -100,
    'bankrupt': -100,
    'fraud': -100,
    'crime': -100,
    'prison': -100,
    'embezzlement': -100,
    'money_trail':-100,
    'jail': -100,
    'sentenced':-100,
    'forfeiture': -100,
    'ill-gotten gain':-100,
    'laundering':-100
}


vader = SentimentIntensityAnalyzer()

vader.lexicon.update(new_words)

In [98]:
def generate_link(identifier_dict, no_of_articles):
    link_start = "https://www.google.com/search?q="
    link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
    link_query = ""
    for identifier in identifier_dict.values():        
        link_query += identifier.replace(' ','+') + '+'
    return link_start + link_query + link_end + "&num=" + str(no_of_articles)

def article_extraction(link):
    article = Article(link)
    article.download()
    try:
        article.parse()
    except:
        pass
    return article.text

def sentiment_analysis(text):
    return vader.polarity_scores(text)


In [99]:
# Main Function
def parse(text):
    #try:
    
    doc = nlp(text)
    tags = [[X.text, X.label_] for X in doc.ents]
    labels = [x.label_ for x in doc.ents]
    items = [x.text for x in doc.ents]

    return tags

def find_names(tags):
    names = []
    for tag in tags:
        if tag[1] == 'PERSON':
            names.append(tag[0])
    return names
    

name_list = ['Lee Yan Ru', 'Lee', 'Yan Ru']
def search_articles_on_individual(individual_dict, no_of_articles):
    def generate_link(identifier_dict, no_of_articles):
        link_start = "https://www.google.com/search?q="
        link_end = "&sxsrf=ALeKk01K1bOuJFHjy4HBARo1cRpUYakYPg:1629640327633&source=lnms&tbm=nws&sa=X&sqi=2&ved=2ahUKEwiu29um48TyAhWGqpUCHYuoAlcQ_AUoAnoECAEQBA&biw=1441&bih=718&dpr=2" 
        link_query = ""
        for identifier in identifier_dict.values():        
            link_query += identifier.replace(' ','+') + '+'
        return link_start + link_query + link_end + "&num=" + str(no_of_articles)
    
    link = generate_link(individual_dict, no_of_articles)
    req = Request(link, headers = {'User-Agent': 'Mozilla/5.0'})

    webpage = urlopen(req).read()

    links = []
    with requests.Session() as c:
        soup = BeautifulSoup(webpage, 'html5lib')
        #print(soup)
        for item in soup.find_all('div', attrs = {'class': "ZINbbc xpd O9g5cc uUPGi"}):
            current_dict = {}
            raw_link = (item.find('a', href = True)['href'])
            link = (raw_link.split("/url?q=")[1]).split('&sa=U&')[0]
            title = (item.find('div',attrs = {'class': 'BNeawe vvjwJb AP7Wnd'})).get_text()
            description  = (item.find('div',attrs = {'class': 'BNeawe s3v9rd AP7Wnd'}).get_text())
            time = description.split(" · ")[0]
            #print(description)
            descript = description.split(" · ")[1]
            current_dict['title'] = title
            current_dict['time'] = time
            current_dict['description'] = descript
            current_dict['link'] = link
            current_dict['text'] = article_extraction(link)
            current_dict['sentiment'] = sentiment_analysis(article_extraction(link))
            
            parsed_description = parse(description)
            names_in_description = find_names(parsed_description)
            # current_dict['desc_names'] = names_in_description
            
            parsed_text = parse(article_extraction(link))
            names_in_text = find_names(parsed_text)
            # current_dict['text_names'] = names_in_text

            names_list = Counter(names_in_description + names_in_text)
            current_dict['names_list'] = names_list
            
            links.append(current_dict)
            
    return links

In [100]:
lee_query = search_articles_on_individual(lee_dict, 10)


In [101]:
df = pd.DataFrame(lee_query)
df

,title,time,description,link,text,sentiment,names_list
0,SMU student found guilty of molesting woman at...,2 weeks ago,"Lee Yan Ru, 25, claimed trial to molesting a w...",https://www.todayonline.com/singapore/smu-stud...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...",{'Lee Yan Ru': 1}
1,'She was completely fine with all my advances'...,5 months ago,SINGAPORE: A 24-year-old man on trial for mole...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: A 24-year-old man on trial for mole...,"{'neg': 0.019, 'neu': 0.864, 'pos': 0.117, 'co...","{'Lee Yan Ru': 2, 'Lee': 34, 'Josephine Chee':..."
2,SMU student grilled on the stand about consent...,5 months ago,Under cross-examination from Deputy Public Pro...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: A Singapore Management University (...,"{'neg': 0.029, 'neu': 0.885, 'pos': 0.087, 'co...","{'Andre Chong': 2, 'Lee Yan Ru': 2, 'Lee': 53,..."
3,SMU student on trial for molesting 22-year-old...,12 months ago,"Lee Yan Ru, 24, allegedly invited a young woma...",https://www.todayonline.com/singapore/smu-stud...,,"{'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound...","{'Lee Yan Ru': 1, 'Netflix': 1}"
4,SMU molest trial: DPP grills accused on issue ...,5 months ago,"... of Lee Yan Ru, 24, who is on trial for out...",https://www.tnp.sg/news/singapore/smu-molest-t...,Singapore Management University student Lee Ya...,"{'neg': 0.066, 'neu': 0.872, 'pos': 0.062, 'co...","{'Lee Yan Ru': 3, 'Andre Chong's': 1, 'Lee': 1..."
5,Man on trial for molest: I don't think she mea...,5 months ago,The Singapore Management University (SMU) stud...,https://www.tnp.sg/news/singapore/man-trial-mo...,Lee Yan Ru is accused of rubbing his exposed p...,"{'neg': 0.051, 'neu': 0.912, 'pos': 0.037, 'co...","{'Lee Yan Ru': 3, 'Lee': 7, 'Josephine Chee': 1}"
6,SMU molestation trial: Accused says victim was...,5 months ago,The Singapore Management University (SMU) stud...,https://stomp.straitstimes.com/singapore-seen/...,"Jean Iau\n\nThe Straits Times\n\nApril 6, 2021...","{'neg': 0.067, 'neu': 0.886, 'pos': 0.046, 'co...","{'Lee Yan Ru': 2, 'Jean Iau ': 1, 'Lee': 10, ..."
7,Woman says she felt angry to be 'disrespected ...,12 months ago,The 24-year-old man was allegedly kneeling ove...,https://www.channelnewsasia.com/singapore/smu-...,SINGAPORE: A woman who was allegedly molested ...,"{'neg': 0.103, 'neu': 0.872, 'pos': 0.025, 'co...","{'Lee Yan Ru's': 2, 'Lee': 11, 'Socially': 1, ..."
8,SMU molest trial: Alleged victim participated ...,12 months ago,Agree or disagree?” asked lawyer Thong Chee Ku...,https://sg.news.yahoo.com/smu-molest-trial-all...,(Yahoo News Singapore file photo)\n\nSINGAPORE...,"{'neg': 0.08, 'neu': 0.868, 'pos': 0.052, 'com...","{'Lee Yan Ru': 2, 'Yan Ru': 2, 'Lee': 14, 'Vic..."
9,NUS student allegedly assaulted woman in campu...,11 months ago,"One of them, Singapore Management University (...",https://www.straitstimes.com/singapore/courts-...,SINGAPORE - A National University of Singapore...,"{'neg': 0.228, 'neu': 0.756, 'pos': 0.016, 'co...","{'Lee Yan Ru': 2, 'Kim Dohyung': 1, 'Kim': 3, ..."


In [ ]:
for header in df['description']:
    print(parse(header))

In [167]:
df.to_excel('lee_query.xlsx')

In [49]:




def article_extraction(link):
    article = Article(link)
    article.download()
    article.parse()
    return article.text

def parse(link):
    doc = nlp(article_extraction(link))
    print([[X.text, X.label_] for X in doc.ents])
    labels = [x.label_ for x in doc.ents]
    print(Counter(labels))
    items = [x.text for x in doc.ents]
    print(Counter(items).most_common(10))
    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/dylan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [194]:
def names_and_ages(tags,age):
    result = []
    try:
        for i in range(len(tags)):
            #if second item is a name
            if tags[i][1] == 'PERSON':
                
                forward_age = forward_searcher(i,tags)
                backwards_age = backward_searcher(i,tags)
                new_list = [tags[i],forward_age,backwards_age]
                #new_list = [tags[i-1],tags[i],tags[i+1]]
                print(new_list)
                
                if detect_age(age,new_list) and tags[i][0] in name_list:

                    print(new_list)
                    #result += new_list
                    
                    if str(age) in new_list[1][0]:
                        print('****************')
                        return [tags[i], new_list[1]]
                    elif str(age) in new_list[2][0]:
                        print('****************')
                        return [tags[i],new_list[2]]
        return result
    except IndexError as e:
        pass 
                

In [182]:
useless_dates = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','yesterday','today']


#index is index of person
def forward_searcher(index,tags):
    for i in range(index,len(tags)):
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
    return [None,None]

def backward_searcher(index,tags):
    i = index
    while i >= 0:
        if tags[i][1] == 'DATE' and tags[i][0] not in useless_dates:
            return tags[i]
        else:
            i -=1

def detect_age(age,lst):
    try:
        if lst[1] is not None and lst[2] is not None:
            date1 = lst[1][0]
            date2 = lst[2][0]
            if (str(age) in date1) or (str(age) in date2):
                return True
        else:

            if lst[1] == None:
                if str(age) in lst[2][0]:
                    return True

            if lst[2] == None:
                if str(age) in lst[1][0]:
                    return True
    except TypeError as e:
        pass
    

In [196]:
for text in df[:6]['text']:
    #print(df.iloc[[index]])
    print(parse(text))
    print("\n")
    
#lee yan ru molest a girl. he is a 24 year old man


[]


[['SINGAPORE', 'ORG'], ['24-year-old', 'DATE'], ['overnight', 'TIME'], ['the Singapore Management University', 'ORG'], ['SMU', 'ORG'], ['Monday', 'DATE'], ['Lee Yan Ru', 'PERSON'], ['fourth-year', 'DATE'], ['SMU', 'ORG'], ['22', 'DATE'], ['Lee', 'PERSON'], ['one', 'CARDINAL'], ['SMU', 'ORG'], ['Jan 8, 2019', 'DATE'], ['Josephine Chee', 'PERSON'], ['Rajah & Tann', 'ORG'], ['Lee', 'PERSON'], ['SMU', 'ORG'], ['early January 2019', 'DATE'], ['Explorer', 'ORG'], ['Instagram', 'ORG'], ['Instagram', 'ORG'], ['Telegram', 'GPE'], ['FIRST', 'ORDINAL'], ['Lee', 'PERSON'], ['first', 'ORDINAL'], ['SMU', 'ORG'], ['Lee', 'PERSON'], ['Lee', 'PERSON'], ['first', 'ORDINAL'], ['Lee', 'PERSON'], ['Telegram', 'ORG'], ['that week', 'DATE'], ['later that night', 'TIME'], ['Lee', 'PERSON'], ['SMU', 'ORG'], ['SMU', 'ORG'], ['Lee', 'PERSON'], ['first', 'ORDINAL'], ['SMU', 'ORG'], ['midnight', 'TIME'], ['Jan 8, 2019', 'DATE'], ['Ms Chee', 'PERSON'], ['Ms Chee', 'PERSON'], ['Lee', 'PERSON'], ['Lee', 'PERSON'

In [145]:
"""
search for nearest name
if 2 nearest name take the correct answer if possible
if not possible = edge case bobian
"""

'\nsearch for nearest name\nif 2 nearest name take the correct answer if possible\nif not possible = edge case bobian\n'

In [195]:
for text in df[:10]['text']:
    #print(df.iloc[[index]])
    #5th article got problem
    #insert the age here
    print(names_and_ages(parse(text),24))
    print("\n")

[]


[['Lee Yan Ru', 'PERSON'], ['fourth-year', 'DATE'], ['24-year-old', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['fourth-year', 'DATE'], ['24-year-old', 'DATE']]
****************
[['Lee Yan Ru', 'PERSON'], ['24-year-old', 'DATE']]


[['Andre Chong', 'PERSON'], ['24-year-old', 'DATE'], None]
[['Lee Yan Ru', 'PERSON'], ['22', 'DATE'], ['24-year-old', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['22', 'DATE'], ['24-year-old', 'DATE']]
****************
[['Lee Yan Ru', 'PERSON'], ['24-year-old', 'DATE']]


[['Lee Yan Ru', 'PERSON'], ['about a week', 'DATE'], ['24-year-old', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['about a week', 'DATE'], ['24-year-old', 'DATE']]
****************
[['Lee Yan Ru', 'PERSON'], ['24-year-old', 'DATE']]


[['Lee Yan Ru', 'PERSON'], ['2019', 'DATE'], None]
[["Andre Chong's", 'PERSON'], ['24', 'DATE'], ['2019', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['2019', 'DATE']]
[['Lee Yan Ru', 'PERSON'], ['24', 'DATE'], ['2019', 'DATE']]
****************
[['Lee Yan Ru', 'PERSON